In [ ]:
import cv2
import os
import threading
import time
import random
from flask import Flask, Response

# Конфигурация сервера
host_name = '0.0.0.0'
port = 8000

# Путь к папке с изображениями
image_folder = r'C:\Users\PC1\Desktop\protocol\test_photo'

# Глобальная переменная для хранения текущего изображения
current_image = None

# Параметры изменения размера
resize_width = 1920  # Ширина изображения после изменения размера
resize_height = 1280  # Высота изображения после изменения размера

app = Flask(__name__)

def update_image():
    global current_image
    while True:
        # Получаем список всех файлов в папке
        image_files = [f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))]
        
        if not image_files:
            print(f"Ошибка: в папке {image_folder} нет изображений.")
        else:
            # Выбираем случайное изображение из списка
            random_image = random.choice(image_files)
            image_path = os.path.join(image_folder, random_image)
            print(f"Чтение случайного изображения из файла: {image_path}")
            img = cv2.imread(image_path)

            if img is not None:
                # Изменение размера изображения
                print(f"Изменение размера изображения до {resize_width}x{resize_height}")
                current_image = cv2.resize(img, (resize_width, resize_height))
                print("Изображение успешно обновлено.")
            else:
                print(f"Ошибка: не удалось прочитать изображение {image_path}. Возможно, файл поврежден.")
        
        # Рандомное время задержки между обновлениями
        time.sleep(random.uniform(0.01, 0.5))

def generate_video():
    global current_image
    while True:
        image = current_image  # Работать с локальной копией current_image
        if image is not None:
            print("Отправка текущего изображения")
            # Конвертация изображения в формат JPEG
            ret, jpeg = cv2.imencode('.jpg', image)
            if ret:
                yield (b'--frame\r\n'
                       b'Content-Type: image/jpeg\r\n\r\n' + jpeg.tobytes() + b'\r\n\r\n')
            else:
                print("Ошибка: не удалось конвертировать изображение в JPEG")
        else:
            print("Ожидание загрузки первого изображения...")
            time.sleep(1 / 20)  # Ждем для 20 FPS

@app.route('/')
def video_feed():
    return Response(generate_video(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    print("Запуск скрипта")

    # Создание и запуск параллельного потока для обновления изображения
    update_thread = threading.Thread(target=update_image)
    update_thread.daemon = True
    update_thread.start()
    print("Параллельный поток для обновления изображения запущен.")
    
    # Запуск Flask-сервера
    app.run(host=host_name, port=port)
